In [1]:
%load_ext autoreload
%autoreload 2

import sys
import time
import os
sys.path.insert(0, '../')

# import embedding methods
from embedding.bernoulli import Bernoulli
from embedding.kl import KL
from embedding.matrix_factorization import MatrixFactorization

# import evaluation methods
from evaluation import evaluate_link_prediction
from evaluation import evaluate_node_classification
from evaluation import evaluate_node_clustering
# 
# import utils
from utils import graph_util
from utils import plot_util

# visualization
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

## Experiment 1

In [6]:
# Experiment 1

exp = {
    "max_epochs": 10000,
    "learning_rate": 1e-2, #Adam
    "weight_decay": 1e-7,
    
    "link_prediction":True,
    "link_pred_num_rounds": 10,
    "link_pred_train_ratio": 0.8,
    "link_pred_eval_every_n_steps": 50,
    "link_pred_edge_emb_method": "average",
    
    "node_classification":False,
    "node_class_num_rounds": 10,
    "node_class_train_ratio": 0.2,
    "node_class_eval_every_n_steps": 50,
    
    "node_clustering": False,
    "node_clustering_num_rounds": 10,
    "node_clustering_eval_epochs": 50,
}


datasets = ["cora"]


similarities= ['adjacency','laplacian','transition', 'sym_normalized_laplacian','NetMF','ppr','sum_power_tran']

for sim in similarities:

    model_1 = MatrixFactorization(embedding_dimension=1, similarity_measure=sim) 
    model_2 = MatrixFactorization(embedding_dimension=2, similarity_measure=sim) 
    model_3 = MatrixFactorization(embedding_dimension=3, similarity_measure=sim) 
    model_4 = MatrixFactorization(embedding_dimension=4, similarity_measure=sim) 
    model_8 = MatrixFactorization(embedding_dimension=8, similarity_measure=sim) 
    model_12 = MatrixFactorization(embedding_dimension=12, similarity_measure=sim) 
    model_16 = MatrixFactorization(embedding_dimension=16, similarity_measure=sim) 
    model_24 = MatrixFactorization(embedding_dimension=24, similarity_measure=sim) 
    model_32 = MatrixFactorization(embedding_dimension=32, similarity_measure=sim) 
    model_64 = MatrixFactorization(embedding_dimension=64, similarity_measure=sim) 
    model_128 = MatrixFactorization(embedding_dimension=128, similarity_measure=sim) 


    embedding_methods = [model_1,model_2,model_3,
                         model_4,model_8,model_12, 
                         model_16, model_24, model_32, 
                         model_64,model_128]
    
# setup folders to store experiment setup summary and results
    result_folder = plot_util.setup_folders_and_summary_files(exp, datasets, embedding_methods)
    print(f'The results of the current experiment are stored at experiments/{result_folder}')

    for dataset in datasets:
        print(f'##### {dataset} #####\n\n')
    
    # load dataset
        A, y = graph_util.load_dataset(dataset)
    
        for model in embedding_methods:
        
            print(model.get_method_summary())
            start = time.time()
        
        # link prediction
            if(exp["link_prediction"]):
                link_prediction_folder = result_folder + "/link_prediction"
                evaluate_link_prediction.expLP(A,dataset,model,exp["link_pred_num_rounds"],
                                            link_prediction_folder, train_ratio=exp["link_pred_train_ratio"], 
                                           edge_emb_method=exp["link_pred_edge_emb_method"],train_epochs=exp["max_epochs"],
                                           eval_epochs=exp["link_pred_eval_every_n_steps"], undirected=True)
                    
            # node classification
            if(exp["node_classification"]):
                node_classification_folder = result_folder + "/node_classification"
                evaluate_node_classification.expNC(A,y,dataset,model,exp["node_class_num_rounds"],
                                               node_classification_folder, train_ratio=exp["node_class_train_ratio"],
                                               train_epochs=exp["max_epochs"],eval_epochs=exp["node_class_eval_every_n_steps"],undirected=True)
            # node clustering
            if(exp["node_clustering"]):
                node_clustering_folder = result_folder + "/node_clustering"
                evaluate_node_clustering.exp_Node_Clustering(A,y,dataset,model,exp["node_clustering_num_rounds"],
                                           node_clustering_folder, train_epochs=exp["max_epochs"],
                                           eval_epochs=exp["node_clustering_eval_epochs"],undirected=True)
            end = time.time()
            print(f'Model evaluation took: {end-start} seconds')

TypeError: __init__() got an unexpected keyword argument 'embedding_option'